<a href="https://colab.research.google.com/github/ecloguehwang/HSS/blob/master/%EC%9E%AC%EC%88%98%EC%9D%B4%ED%9B%84%EC%84%B1%EC%A0%81%EB%B9%84%EA%B5%90_%EC%8B%A0%EB%AA%A9_RPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리 부르기
import pandas as pd
import numpy as np
import csv


##  데이터 부르기 : 구글colab에 마운트(mount)하는 방법: 왼쪽 폴더아이콘을 누르기-눈동자 왼쪽 파일마운트 아이콘 클릭- /content/drive/MyDrive/python폴더의 susi_2021simple.csv파일을 불러옴
df1 = pd.read_csv('/content/drive/MyDrive/python/sinmok/susi_jsi_sinmok.csv', encoding = 'cp949')
df2 = pd.read_csv('/content/drive/MyDrive/python/sinmok/jesu.csv', encoding = 'cp949')

# 첫 5줄을 봅니다.
df1.tail(4)
#df2.head(4)

In [ ]:
#2021년 고3인원 318명
df1_u = df1['이름'].unique()
len(df1_u)

318

In [ ]:
#2021년 고3인원 318명
df1_u = df1['이름'].unique()
print(df1_u)

In [ ]:
#졸업자 중 2022년에 재수한 학생들
df2_u = df2['이름'].unique()
print(df2_u)

In [ ]:
#2개의 파일을 합침: 이름, 성별 파일과 이름과 여러변수가 있는 파일
df_rj = pd.merge(df1, df2, left_on='이름', right_on='이름', how='right')
df_rj.head(2)

In [ ]:
#합친 파일의 변수별 결측값 수
df_rj.isnull().sum()

In [49]:
#합친 파일을 엑셀로 저장
import openpyxl

#2개의 파일을 합침: 이름, 성별 파일과 이름과 여러변수가 있는 파일
df_rj = pd.merge(df1, df2, left_on='이름', right_on='이름', how='right')


#데이터 프레임으로 전환
df_rj = pd.DataFrame(df_rj)


#결과를 엑셀로 저장
path = '/content/drive/MyDrive/python/sinmok/'
df_rj.to_excel(excel_writer= f'{path}jesu_compare_sinmok.xlsx')



In [ ]:
#재수생수: 234명: 방법1
#지원자수
unique_names = df_rj['이름'].unique()
num_unique_names = len(unique_names)
print(num_unique_names)

234


In [ ]:
#총지원자수: 방법2
unique_names = df_rj['이름'].nunique()
print(unique_names)

234


In [ ]:
#이름으로 표준점수와 백분위 조회
name = input("이름:")
df_name = df_rj[(df_rj.이름 == name)]
df_name = df_name[['이름', '백분위_평균', '백분위평균_졸업', '표점합', '표점합_졸업']]
df_name

In [ ]:
# '백분위_평균' 변수(열) 값 중 결측값 제외
df_rj_restudy = df_rj.dropna(subset=['백분위_평균'])
df_rj_restudy

In [ ]:
# Convert columns to numeric, setting errors='coerce' to turn non-convertible values to NaN
df_rj['백분위_평균'] = pd.to_numeric(df_rj['백분위_평균'], errors='coerce')
df_rj['백분위평균_졸업'] = pd.to_numeric(df_rj['백분위평균_졸업'], errors='coerce')

# Optionally, handle NaN values here (e.g., df_rj.dropna() to remove, or df_rj.fillna(value) to replace them)

# Calculate the mean
mean_백분위_평균 = df_rj['백분위_평균'].mean().round(1)
mean_백분위_평균_졸업 = df_rj['백분위평균_졸업'].mean().round(1)

dif = mean_백분위_평균_졸업 - mean_백분위_평균
dif = dif.round(1)


print(f"고3(2021)이 1년후 재수(2022)했을때 백분위 평균차이가 {dif}%입니다.")
mean_백분위_평균, mean_백분위_평균_졸업

In [ ]:
#특정등급 이상되는 학생의 백분위점수 조회하기 : 이름없는 버전

#라이브러리 불러오기
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import re


#df_rj = df_rj[~df_rj['지원시기'].str.contains('[가나다]', regex=True)]

#입력받기
level_mini = float(input("백분위 점수 몇% 이상 학생을 조회하고 싶나요?:"))
level_max = float(input("백분위 점수 몇% 이하 학생을 조회하고 싶나요?:"))


# Convert columns to numeric, setting errors='coerce' to turn non-convertible values to NaN
df_rj['백분위_평균'] = pd.to_numeric(df_rj['백분위_평균'], errors='coerce')
df_rj['백분위평균_졸업'] = pd.to_numeric(df_rj['백분위평균_졸업'], errors='coerce')


# 수시결과만 출력: 정시 '가나다' 군 제외: Apply str.contains on '지원시기', replacing NaN with an empty string to avoid type errors
df_rj = df_rj[~df_rj['지원시기'].fillna('').str.contains('[가나다]+', regex=True, na=False)]


#이상, 이하 정의
df_rj_nesin = df_rj[(df_rj['백분위_평균'] >= level_mini) & (df_rj['백분위_평균'] <= level_max)]


#필요한 열 모으기
df_rj_nesin = df_rj_nesin[['지원시기', '대학', '모집단위', '최종', '내신_전과목', '백분위_평균', '백분위평균_졸업']].sort_values(by = '내신_전과목')


#내신_전과목의 소수점 첫째자리
df_rj_nesin['내신_전과목'] = df_rj_nesin['내신_전과목'].round(1)


# Convert DataFrame to a list of lists for tabulate
table_data = df_rj_nesin.values.tolist()

# Define the table headers
table_headers = ['지원시기', '대학', '모집단위',  '최종', '내신_전과목', '백분위_평균', '백분위평균_졸업']

# Create the formatted table using tabulate
table_string = tabulate(table_data, headers=table_headers, tablefmt='plain')

print("**********************************************")
print(table_string)

#엑셀로 저장
path = '/content/drive/MyDrive/python/sinmok/result_restudy/'
df_rj_nesin.to_excel(excel_writer= f'{path}고3수능백분위{level_mini}%이상{level_max}%이하.xlsx')


In [ ]:
#특정등급 이상되는 학생의 백분위점수 조회하기 : 이름있는 버전


#라이브러리 불러오기
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import re



#입력받기
level_mini = float(input("백분위 점수 몇% 이상 학생을 조회하고 싶나요?:"))
level_max = float(input("백분위 점수 몇% 이하 학생을 조회하고 싶나요?:"))


# Convert columns to numeric, setting errors='coerce' to turn non-convertible values to NaN
df_rj['백분위_평균'] = pd.to_numeric(df_rj['백분위_평균'], errors='coerce')
df_rj['백분위평균_졸업'] = pd.to_numeric(df_rj['백분위평균_졸업'], errors='coerce')

# 수시결과만 출력: 정시 '가나다' 군 제외: Apply str.contains on '지원시기', replacing NaN with an empty string to avoid type errors
#df_rj = df_rj[~df_rj['지원시기'].fillna('').str.contains('[가나다]+', regex=True, na=False)]


#특정 백분위 이상, 특정 백분위 이하 정의
df_rj_nesin = df_rj[(df_rj['백분위_평균'] >= level_mini) & (df_rj['백분위_평균'] <= level_max)]


df_rj_nesin = df_rj_nesin[['이름', '지원시기', '대학', '모집단위', '최종', '내신_전과목', '백분위_평균', '백분위평균_졸업']].sort_values(by = '내신_전과목')

df_rj_nesin['내신_전과목'] = df_rj_nesin['내신_전과목'].round(1)

# Convert DataFrame to a list of lists for tabulate
table_data = df_rj_nesin.values.tolist()

# Define the table headers
table_headers = ['이름', '지원시기', '대학', '모집단위',  '최종', '내신_전과목', '백분위_평균', '백분위평균_졸업']

# Create the formatted table using tabulate
table_string = tabulate(table_data, headers=table_headers, tablefmt='plain')

print("**********************************************")
print(table_string)

#엑셀로 저장
path = '/content/drive/MyDrive/python/sinmok/result_restudy/'
df_rj_nesin.to_excel(excel_writer= f'{path}고3수능백분위{level_mini}%이상{level_max}%이하.xlsx')


In [ ]:
#특정등급 이상되는 학생들의 수능점수 조회

#라이브러리 불러오기
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import re


#df_rj = df_rj[~df_rj['지원시기'].str.contains('[가나다]', regex=True)]


#입력받기
level_mini = float(input("백분위 점수 몇% 이상 학생을 조회하고 싶나요?:"))
level_max = float(input("백분위 점수 몇% 이하 학생을 조회하고 싶나요?:"))



# 수시결과만 출력: 정시 '가나다' 군 제외: Apply str.contains on '지원시기', replacing NaN with an empty string to avoid type errors
df_rj = df_rj[~df_rj['지원시기'].fillna('').str.contains('[가나다]+', regex=True, na=False)]


df_rj_nesin = df_rj[(df_rj['백분위_평균'] >= level_mini) & (df_rj['백분위_평균'] < level_max)]


#이름 중복자 제거
df_rj_nesin = df_rj_nesin.drop_duplicates('이름')

#열 리스트
list = ['지원시기', '선택_국어', '백분위_국어', '선택_수학', '백분위_수학',
        '영어등급', '백분위_탐구1', '백분위_탐구2', '백분위_평균']


#백분위평균으로 오름차순 정렬
df_rj_nesin = df_rj_nesin[list].sort_values(by = '백분위_평균')


df_rj_nesin['백분위_평균'] = df_rj_nesin['백분위_평균'].round(1)


# Convert DataFrame to a list of lists for tabulate
table_data = df_rj_nesin.values.tolist()

# Define the table headers
table_headers = ['지원시기','선택_국어', '백분위_국어', '선택_수학', '백분위_수학',
        '영어등급', '백분위_탐구1', '백분위_탐구2', '백분위_평균']

# Create the formatted table using tabulate
table_string = tabulate(table_data, headers=table_headers, tablefmt='plain')

print("**********************************************")
print(table_string)

#엑셀로 저장
path = '/content/drive/MyDrive/python/sinmok/result_restudy/'
df_rj_nesin.to_excel(excel_writer= f'{path}고3수능백분위{level_mini}%이상{level_max}%이하 학생의 과목별 백분위점수.xlsx')


In [ ]:
#수학과목 선택 빈도
df_rj.groupby('선택_수학').count()

In [14]:
df_rj['백분위_평균'].isnull().sum()

107

In [13]:
# 특정 '문자열'을 포함하는 열(column)을 뽑아내는 코드:
math_columns = [col for col in df_rj if '백분위_평균' in col]
print(math_columns)

['백분위_평균']
